## Setup

In [1]:
# imports

import tensorflow as tf
from tensorflow import keras
from keras.layers import (
    Flatten,
    Dense,
) 
from keras.applications.vgg16 import VGG16
from PIL import Image
import numpy as np

In [2]:
# check cpu and gpu

num_cpus = len(tf.config.list_physical_devices('CPU'))
num_gpus = len(tf.config.list_physical_devices('GPU'))

if num_cpus > 0:
    print("CPU available. ")
else:
    print("No CPU available. ")

if num_gpus > 0:
    print("GPU available. ")
else:
    print("No GPU available. ")

CPU available. 
No GPU available. 


## Data preparation

In [3]:
# create a dictionary of images and their lists of bounding boxes

rows = open("data/data.csv").read().strip().split("\n")

images_and_box_lists = {}

current_image = "images/0.png"
temp_box_list = []
for row in rows: 
    row = row.split(",")

    if(row[0] != current_image): 
        images_and_box_lists.update({current_image : temp_box_list})
        current_image = row[0]
        temp_box_list = []
    temp_box_list.append([row[1], row[2], row[3], row[4]])

In [5]:
# split images and make labels list

split_images = []
labels = []

w = 320
h = 360

for key in images_and_box_lists:
    full_image = Image.open("data/" + key)

    # loop through boxes
    for window_y1 in range(0, 1800, h):
        for window_x1 in range(0, 2880, w):
            window_x2 = window_x1 + w
            window_y2 = window_y1 + h

            cropped_image = full_image.crop(window_x1, window_y1, window_x2, window_y2)
            label = 0
            
            # iou for all balloon boxes in list
            for balloon_boxes in images_and_box_lists.get(key):
                balloon_x1 = balloon_boxes[0]
                balloon_y1 = balloon_boxes[1]
                balloon_w = balloon_boxes[2]
                balloon_h = balloon_boxes[3]

                balloon_x2 = balloon_x1 + balloon_w
                balloon_y2 = balloon_y1 + balloon_h

                intersection_x1 = max(window_x1, balloon_x1)
                intersection_y1 = max(window_y1, balloon_y1)
                intersection_x2 = min(window_x2, balloon_x2)
                intersection_y2 = min(window_y2, balloon_y2)

                intersection_area = max(0, intersection_x2 - intersection_x1) * max(0, intersection_y2 - intersection_y1)

                window_area = w * h
                balloon_area = balloon_w * balloon_h

                if intersection_area / (window_area + balloon_area - intersection_area) > 0.5:
                    label = 1
                    break;	            
            
            split_images.append(cropped_image)
            labels.append()

In [ ]:
# draw labels for split images

## Different network models

In [1]:
# vgg16

vgg = VGG16(
    weights="imagenet",
    include_top=False,
    input_tensor=keras.layers.Input(shape=(32, 32, 3)),
)

vgg.trainable = False

window_x1 = vgg.output
window_x1 = Flatten()(window_x1)
window_x1 = Dense(128, activation="relu")(window_x1)
window_x1 = Dense(64, activation="relu")(window_x1)
window_x1 = Dense(32, activation="relu")(window_x1)
window_x1 = Dense(1, activation="sigmoid")(window_x1)
model = keras.Model(inputs=vgg.input, outputs=window_x1)

SyntaxError: invalid syntax (1342156391.py, line 6)

## Network building

In [41]:
# split data

train_images, test_images = np.split(images, [int(len(images) * 0.8)])
train_boxes, test_boxes = np.split(boxes, [int(len(boxes) * 0.8)])

In [42]:
# train

model.compile(
    loss=keras.losses.MeanSquaredError(reduction="sum_over_batch_size", name="mse"),
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
)

model.fit(
    train_images,
    train_boxes,
    epochs=20,
    validation_data=(test_images, test_boxes),
    verbose=1,
    shuffle=True,
)

Epoch 1/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 154s 3s/step - loss: 0.0713 - val_loss: 0.0385
Epoch 2/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 157s 3s/step - loss: 0.0316 - val_loss: 0.0338
Epoch 3/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 144s 3s/step - loss: 0.0292 - val_loss: 0.0332
Epoch 4/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 126s 3s/step - loss: 0.0281 - val_loss: 0.0333
Epoch 5/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 133s 3s/step - loss: 0.0296 - val_loss: 0.0332
Epoch 6/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 128s 3s/step - loss: 0.0276 - val_loss: 0.0332
Epoch 7/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 110s 2s/step - loss: 0.0266 - val_loss: 0.0329
Epoch 8/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 101s 2s/step - loss: 0.0271 - val_loss: 0.0328
Epoch 9/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 1402s 29s/step - loss: 0.0273 - val_loss: 0.0327
Epoch 10/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 166s 3s/step - loss: 0.0262 - val_loss: 0.0335
Epoch 11/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 176s 4s/step - loss: 0.0275 - val_loss: 0.0331
Epoch 12/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 162s 3s/step - loss: 0

In [43]:
# test

model.evaluate(
    test_images,
    test_boxes,
    verbose=1
)

13/13 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - loss: 0.0323


0.03264449164271355

In [44]:
# save

model.save("models/vgg_good_20_20.keras")